<a href="https://colab.research.google.com/github/smkmohsin/Drowsiness-Detection/blob/main/Drowsiness_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dlib
!pip install face_recognition

     |████████████████████████████████| 100.1 MB 24 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=df258545fc218e6579a774ac2b55089767a449560e76ace24fdee6b6e3a0b86d
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [3]:
import cv2
import face_recognition as fr
import dlib
import os

# **Basic Face Matching**

First, we get the location of where exactly the face is in the image using face_location() method(which gets the outline of the face) on the RGB image. Then face encodings(markings of eyes, nose, mouth, jaws which remain the same for different images of the same person) are taken using face_encodings() function which returns a list containing 128 measurements. Both these two steps are followed for the original and test image. Then a comparison between these two returned lists is done by the function compare_faces() which returns a list of boolean values(True or False). The face distance function gets the value of that by how much the two images differ. The lower the distance the better the matching and vice versa.

In [5]:
# With same image
imgMoh = fr.load_image_file('/content/drive/MyDrive/Almabetter/Module 7/Capstone Project/ImagesAttendance/Mohsin.PNG')
Test1 = imgMoh
fLoc = fr.face_locations(imgMoh)[0]
encodeMoh = fr.face_encodings(imgMoh)[0]
fLocTest = fr.face_locations(Test1)[0]
encTest = fr.face_encodings(Test1)[0]
result1 = fr.compare_faces([encodeMoh],encTest)
faceDist1 = fr.face_distance([encodeMoh],encTest)
print(result1,faceDist1)

# With different image
imgMoh = fr.load_image_file('/content/drive/MyDrive/Almabetter/Module 7/Capstone Project/ImagesAttendance/Mohsin.PNG')
Test2 = fr.load_image_file('/content/drive/MyDrive/Almabetter/Module 7/Capstone Project/ImagesAttendance/Chris.jpg')
fLoc = fr.face_locations(imgMoh)[0]
encodeMoh = fr.face_encodings(imgMoh)[0]
fLocTest = fr.face_locations(Test2)[0]
encTest = fr.face_encodings(Test2)[0]
result2 = fr.compare_faces([encodeMoh],encTest)
faceDist2 = fr.face_distance([encodeMoh],encTest)
print(result2,faceDist2)

[True] [0.]
[False] [1.02709267]


# Building Face Attendance System 

Now we are ready to build a realtime face attendance system wherein webcam captured frames will be matched against the existing database images and if the match is found then it’ll store it in a CSV file called ‘Attendance Register’ along with name and time of capture. Only once the file will store the matched image’s details, if the same image is received again then it’ll not update.

Path setting to the directory containing the image database. Read each image and the images array. Append the filenames into a list called Names and remove the extension.

In [6]:
pathlib = '/content/drive/MyDrive/Almabetter/Module 7/Capstone Project/ImagesAttendance'
images = []
Names = []
myList = os.listdir(pathlib)
print(myList)
for cl in myList:
    currImg = cv2.imread(f'{pathlib}/{cl}')
    images.append(currImg)
    Names.append(os.path.splitext(cl)[0])
print(Names)

['Chris.jpg', 'Mohsin.PNG']
['Chris', 'Mohsin']


Finding face encodings of images in the database and keeping them in a list to use later with incoming frames. 

In [7]:
def DbEncodings(images):
    encList = []
    for image in images:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        enc = fr.face_encodings(image)[0]
        encList.append(enc)
    return encodeList

In [8]:
# Capturing video frames

cap = cv2.VideoCapture(0)

# Iterating through frames

while True:
    _, img = cap.read()
    # image = cv2.resize(img,(0,0),None,0.25,0.25)
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# The same process is followed by the first detection face location then getting the face encoding values.

    facesInFrame = fr.face_locations(image)
    encodesInFrame = fr.face_encodings(image,facesInFrame)

# Now the incoming images are tested against the previously-stored encodings. Then the face distance is also computed. Lastly, we call the Attendance function along with the person name who is identified.

for encodeFace,faceLoc in zip(encodesInFrame,facesInFrame):
        matchList = fr.compare_faces(encodeKnown,encFace)
        faceDist = fr.face_distance(encodeKnown,encFace)
        match = np.argmin(faceDist)
        if matchList[match]:
            name = Names[match].upper()
            Attendance(name)


#Reading from attendance file, Storing data(Name and Time of entry) if previously not stored.

def Attendance(name):
    with open('Attendance_Register.csv','r+') as f:
        DataList = f.readlines()
        names = []
        for data in DataList:
            ent = data.split(',')
            names.append(ent[0])
        if name not in names:
            curr = datetime.now()
            dt = curr.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dt}')
encodeKnown = DbEncodings(images)
print('Encoding Complete')

error: ignored